In [1]:
import itertools

class Symbol:
    def __init__(self, name):
        self.name = name
    def formula(self):
        return self.name
    def __repr__(self):
        return self.name

class Not:
    def __init__(self, operand):
        self.operand = operand
    def formula(self):
        return f"¬({self.operand.formula()})"
    def __repr__(self):
        return self.formula()

class And:
    def __init__(self, *args):
        self.args = args
    def formula(self):
        return "(" + " ∧ ".join(arg.formula() for arg in self.args) + ")"
    def __repr__(self):
        return self.formula()

class Or:
    def __init__(self, *args):
        self.args = args
    def formula(self):
        return "(" + " ∨ ".join(arg.formula() for arg in self.args) + ")"
    def __repr__(self):
        return self.formula()

class Implication:
    def __init__(self, premise, conclusion):
        self.premise = premise
        self.conclusion = conclusion
    def formula(self):
        return f"({self.premise.formula()} → {self.conclusion.formula()})"
    def __repr__(self):
        return self.formula()

def symbols(sentence):
    """Return all symbols appearing in a logical sentence."""
    if isinstance(sentence, Symbol):
        return {sentence}
    elif isinstance(sentence, (Not, And, Or, Implication)):
        s = set()
        for attr in vars(sentence).values():
            if isinstance(attr, tuple) or isinstance(attr, list):
                for a in attr:
                    s |= symbols(a)
            elif isinstance(attr, (Symbol, Not, And, Or, Implication)):
                s |= symbols(attr)
        return s
    return set()

def evaluate(sentence, model):
    """Evaluate a logical sentence given a model."""
    if isinstance(sentence, Symbol):
        return model[sentence]
    elif isinstance(sentence, Not):
        return not evaluate(sentence.operand, model)
    elif isinstance(sentence, And):
        return all(evaluate(arg, model) for arg in sentence.args)
    elif isinstance(sentence, Or):
        return any(evaluate(arg, model) for arg in sentence.args)
    elif isinstance(sentence, Implication):
        return (not evaluate(sentence.premise, model)) or evaluate(sentence.conclusion, model)
    raise ValueError("Invalid sentence")

def model_check(knowledge, query):
    """Check if knowledge base entails query."""
    all_symbols = list(symbols(And(knowledge, query)))
    for values in itertools.product([True, False], repeat=len(all_symbols)):
        model = dict(zip(all_symbols, values))
        if evaluate(knowledge, model):
            if not evaluate(query, model):
                return False
    return True


In [3]:
# Import everything from logic.py (as per your environment)

# Define propositional symbols
rain = Symbol("rain")             # It is raining
umbrella = Symbol("umbrella")     # You have an umbrella
outside = Symbol("outside")       # You go outside
wet = Symbol("wet")               # You get wet

# Define logical sentences (knowledge base rules)

# If it rains and you don't have an umbrella, then you get wet
rule1 = Implication(And(rain, Not(umbrella)), wet)


# If it rains and you have an umbrella, you can still go outside
rule2 = Implication(And(rain, umbrella), outside)

# If it doesn't rain, you can go outside
rule3 = Implication(Not(rain), outside)

# If you are outside and it rains without an umbrella, you get wet
rule4 = Implication(And(outside, And(rain, Not(umbrella))), wet)

# Combine all into a knowledge base
knowledge_base = And(rule1, rule2, rule3, rule4, umbrella)

# Print formulas for clarity
print("Knowledge Base Formula:")
print(knowledge_base.formula())
print()

# Define the query: does the knowledge base entail that we go outside?
query = outside

# Perform model checking
result = model_check(knowledge_base, query)

# Print result
print(f"Does the knowledge base entail '{query}'? → {result}")


Knowledge Base Formula:
(((rain ∧ ¬(umbrella)) → wet) ∧ ((rain ∧ umbrella) → outside) ∧ (¬(rain) → outside) ∧ ((outside ∧ (rain ∧ ¬(umbrella))) → wet) ∧ umbrella)

Does the knowledge base entail 'outside'? → True
